## hard voting

In [1]:
import os
from os import path as osp
import pandas as pd
#import numpy as np
from tqdm import tqdm

In [2]:
CLASSES = [ 
    'Background', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass',
    'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing'
]

In [3]:
base_dir = ''
target1 = 'adapter_v1_7418.csv' # best model
target2 = 'adapter_v2_7245.csv'
# target3 = 'hornet_v2_7235.csv'
target4 = 'semask_v1_7331.csv'

df1 = pd.read_csv(osp.join(base_dir, target1))
df2 = pd.read_csv(osp.join(base_dir, target2))
# df3 = pd.read_csv(osp.join(base_dir, target3))
df4 = pd.read_csv(osp.join(base_dir, target4))

submission = pd.DataFrame()
submission['image_id'] = df1['image_id']

In [4]:
PredictionString = []

for idx in tqdm(range(len(df1))):
    result1 = df1['PredictionString'][idx].split(' ')
    result2 = df2['PredictionString'][idx].split(' ')
    # result3 = df3['PredictionString'][idx].split(' ')
    result4 = df4['PredictionString'][idx].split(' ')
    result = ''

    for i in range(len(result1)):
        pixel_count = {str(i):0 for i in range(11)}
        best_pixel = result1[i]
        pixel_count[result1[i]] += 1
        pixel_count[result2[i]] += 1
        # pixel_count[result3[i]] += 1
        pixel_count[result4[i]] += 1
        
        voted_pixel = [key for key, value in pixel_count.items() if value == max(pixel_count.values())]
        
        # voted_pixel이 2개 이상인 경우
        if len(voted_pixel) > 1:
            # best_pixel이 뽑은 pixel이 있는 경우
            if best_pixel in pixel_count:
                result += best_pixel + ' '
        # 1개만 뽑힌 경우 (model이 4개 이하일 경우만)
        else:
            result += voted_pixel[0] + ' '
            
    ensemble_result = result[:-1]
    PredictionString.append(ensemble_result)

100%|██████████| 819/819 [09:01<00:00,  1.51it/s]


In [5]:
submission['PredictionString'] = PredictionString
submission.to_csv('./hard_voted_output.csv', index=False)